In [ ]:
import pandas as pd
import numpy as np
import h5py
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from sklearn.metrics import roc_auc_score
import time
import copy
from tqdm.auto import tqdm
import torch.optim as optim
import os

import random

#trying to ensure reproducibility
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [2]:
experiment_num = 15

In [ ]:
#Loading data
with h5py.File('../../data/3d_array/mod_train_data_3d_h5.h5', 'r') as f:
    train_X = f['train_data_3d'][:]
with h5py.File('../../data/3d_array/mod_val_data_3d_h5.h5', 'r') as f:
    val_X = f['val_data_3d'][:]
# with h5py.File('../../data/3d_array/test_data_3d_h5.h5', 'r') as f:
#     test_X = f['test_data_3d'][:]

train_y = pd.read_parquet('../../data/3d_array/train_targets.parquet')
val_y = pd.read_parquet('../../data/3d_array/val_targets.parquet')

In [ ]:
train_X = np.nan_to_num(train_X, nan = 0.0)
val_X = np.nan_to_num(val_X, nan = 0.0)

In [4]:
train_y['end_of_month'].value_counts()

end_of_month
2017-03-31    270223
2017-04-30    270223
2017-05-31    270223
2017-06-30    270223
2017-07-31    270223
2017-08-31    270223
2017-09-30    270223
2017-10-31    270223
2017-11-30    270223
2017-12-31    270223
Name: count, dtype: int64

In [ ]:
train_y = train_y[train_y['end_of_month'].isin(['2018-03-31'])]
val_y = val_y[val_y['end_of_month'].isin(['2018-03-31'])]

C:\Users\vaseekaranv\AppData\Local\Temp\ipykernel_23404\1552306949.py:1: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train_y = train_y[train_y['end_of_month'].isin(['2017-12-31'])]
C:\Users\vaseekaranv\AppData\Local\Temp\ipykernel_23404\1552306949.py:2: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  val_y = val_y[val_y['end_of_month'].isin(['2017-12-31'])]


In [6]:
train_y.sort_values(by=['customer_ID'])

,customer_ID,end_of_month,target
9,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-12-31,0
19,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2017-12-31,0
29,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2017-12-31,0
39,000084e5023181993c2e1b665ac88dbb1ce9ef621ec537...,2017-12-31,0
49,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-12-31,0
...,...,...,...
2702189,ffff39cc22a375d07369980d02d617883dd28ad81a6aa3...,2017-12-31,0
2702199,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,2017-12-31,0
2702209,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,2017-12-31,0
2702219,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,2017-12-31,0


In [7]:
train_X.shape, train_y.shape

((270223, 10, 115), (270223, 3))

In [8]:
val_X.shape, val_y.shape

((115811, 10, 115), (115811, 3))

In [ ]:
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

In [10]:
class SmallRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, fc_size, output_size=1, conv_out_channels=32, kernel_size=3, reduction=16):
        super(SmallRNNModel, self).__init__()
        # Conv1D layer
        self.conv1d = nn.Conv1d(in_channels=input_size, out_channels=conv_out_channels, kernel_size=kernel_size, padding=kernel_size//2)
        self.relu = nn.ReLU()
        # SEBlock
        self.se_block = SEBlock(channel=conv_out_channels, reduction=reduction)
        # LSTM layer
        self.lstm = nn.LSTM(conv_out_channels, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, fc_size)
        self.output = nn.Linear(fc_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Input shape: batch_size x time_steps x features
        # Conv1D expects: batch_size x features(channels) x time_steps
        x = x.permute(0, 2, 1)  # Rearrange for Conv1D
        # Apply Conv1D
        x = self.conv1d(x)
        x = self.relu(x)
        # Apply SEBlock
        x = self.se_block(x)
        # Rearrange back for LSTM: batch_size x time_steps x features
        x = x.permute(0, 2, 1)  
        # LSTM layer
        lstm_out, _ = self.lstm(x)
        # Take the output of the last time step
        lstm_last_out = lstm_out[:, -1, :]
        # Fully connected layer
        fc_out = self.fc(lstm_last_out)
        # Final output layer
        output = self.output(fc_out)
        # Apply sigmoid for binary classification
        return self.sigmoid(output)

In [11]:
# Example usage
input_size = train_X.shape[2]  # Number of features
hidden_size = 64  # Hidden state size for LSTM
fc_size = 32  # Size of the fully connected layer
conv_out_channels = 32  # Number of output channels for Conv1D
kernel_size = 3  # Kernel size for Conv1D

model = SmallRNNModel(
    input_size=input_size, 
    hidden_size=hidden_size, 
    fc_size=fc_size,
    conv_out_channels=conv_out_channels,
    kernel_size=kernel_size
)

In [12]:
batch_size = 10000
from torchinfo import summary
summary(model, input_size=(batch_size, train_X.shape[1], train_X.shape[2]), device='cpu',
        col_names=["input_size", "kernel_size","output_size", "num_params"])

Layer (type:depth-idx)                   Input Shape               Kernel Shape              Output Shape              Param #
SmallRNNModel                            [10000, 10, 115]          --                        [10000, 1]                --
├─Conv1d: 1-1                            [10000, 115, 10]          [3]                       [10000, 32, 10]           11,072
├─ReLU: 1-2                              [10000, 32, 10]           --                        [10000, 32, 10]           --
├─SEBlock: 1-3                           [10000, 32, 10]           --                        [10000, 32, 10]           --
│    └─AdaptiveAvgPool1d: 2-1            [10000, 32, 10]           --                        [10000, 32, 1]            --
│    └─Sequential: 2-2                   [10000, 32]               --                        [10000, 32]               --
│    │    └─Linear: 3-1                  [10000, 32]               --                        [10000, 2]                64
│    │    └─ReL

In [13]:
from torch.utils.data import Dataset, DataLoader
class TimeSeriesDataset(Dataset):
    def __init__(self, data, targets):
        """
        Args:
            data: numpy array of shape (num_ids, time_steps, features)
            targets: numpy array of shape (num_ids,)
        """
        self.data = torch.FloatTensor(data)
        self.targets = torch.FloatTensor(targets).unsqueeze(1)  # Add dimension for output
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]

In [14]:
train_dataset = TimeSeriesDataset(train_X, train_y['target'].values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [15]:
val_dataset = TimeSeriesDataset(val_X, val_y['target'].values)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [16]:
train_dataset.__getitem__(0)[0].shape, train_dataset.__getitem__(0)[1]

(torch.Size([10, 115]), tensor([0.]))

In [17]:
val_dataset.__getitem__(0)[0].shape, val_dataset.__getitem__(0)[1]

(torch.Size([10, 115]), tensor([0.]))

In [18]:
from sklearn.metrics import roc_auc_score
import time
import copy
from tqdm.auto import tqdm

import torch.optim as optim

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training parameters
num_epochs = 20
patience = 3  # Number of epochs to wait for improvement before early stopping

# Initialize variables for early stopping
best_val_loss = float('inf')
best_val_auc = 0.0
best_model_wts = copy.deepcopy(model.parameters)
no_improve_epochs = 0

# For tracking metrics
train_losses = []
val_losses = []
val_aucs = []

# Move model to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"Training on {device}")
start_time = time.time()

# Training loop

# Modify the training loop to include progress bars
for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    
    # Add tqdm progress bar for training loop
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
    for inputs, labels in train_pbar:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        
        # Update progress bar with current loss
        train_pbar.set_postfix({'loss': loss.item()})
    
    epoch_train_loss = running_loss / len(train_dataset)
    train_losses.append(epoch_train_loss)
    
    # Validation phase
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    
    # Add tqdm progress bar for validation loop
    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Valid]", leave=False)
    with torch.no_grad():
        for inputs, labels in val_pbar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * inputs.size(0)
            
            # Collect predictions and labels for AUC calculation
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            # Update progress bar with current loss
            val_pbar.set_postfix({'loss': loss.item()})
    
    # Calculate metrics
    epoch_val_loss = running_loss / len(val_dataset)
    val_losses.append(epoch_val_loss)
    
    all_preds = [p[0] for p in all_preds]  # Flatten predictions
    all_labels = [l[0] for l in all_labels]  # Flatten labels
    epoch_val_auc = roc_auc_score(all_labels, all_preds)
    val_aucs.append(epoch_val_auc)
    
    # Print epoch statistics
    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {epoch_train_loss:.4f}, "
          f"Val Loss: {epoch_val_loss:.4f}, "
          f"Val AUC: {epoch_val_auc:.4f}")
    
    # Check if this is the best model
    if epoch_val_loss < best_val_loss:
        best_val_loss = epoch_val_loss
        best_val_auc = epoch_val_auc
        best_model_wts = copy.deepcopy(model.state_dict())
        no_improve_epochs = 0
    else:
        no_improve_epochs += 1
    
    # Early stopping
    if no_improve_epochs >= patience:
        print(f"Early stopping triggered after {epoch+1} epochs")
        break


# Training complete
time_elapsed = time.time() - start_time
print(f"Training completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s")
print(f"Best val loss: {best_val_loss:.4f}, Best val AUC: {best_val_auc:.4f}")

# Load best model weights
model.load_state_dict(best_model_wts)

Training on cuda:0


Epoch 1/20 [Train]:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 1/20 [Valid]:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/20 - Train Loss: 0.5465, Val Loss: 0.3431, Val AUC: 0.9004


Epoch 2/20 [Train]:   0%|          | 0/28 [00:00<?, ?it/s]

...

Epoch 16/20 - Train Loss: 0.2520, Val Loss: 0.2556, Val AUC: 0.9418


Epoch 17/20 [Train]:   0%|          | 0/28 [00:00<?, ?it/s]

Epoch 17/20 [Valid]:   0%|          | 0/12 [00:00<?, ?it/s]

Epoch 17/20 - Train Loss: 0.2513, Val Loss: 0.2566, Val AUC: 0.9418
Early stopping triggered after 17 epochs
Training completed in 1m 34s
Best val loss: 0.2555, Best val AUC: 0.9417


<All keys matched successfully>

In [19]:
import os

# Save the model weights

# Create directory if it doesn't exist
save_dir = '../../models/deep_learning'
os.makedirs(save_dir, exist_ok=True)

# Save model state dictionary
model_path = os.path.join(save_dir, f'experiment_{experiment_num}.pth')
torch.save(model.state_dict(), model_path)

# Save additional information for later reference
checkpoint_path = os.path.join(save_dir, f'experiment_{experiment_num}.pth')
checkpoint = {
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}
torch.save(checkpoint, checkpoint_path)

print(f"Model saved to {model_path}")
print(f"Checkpoint saved to {checkpoint_path}")

Model saved to ../../models/deep_learning\experiment_15.pth
Checkpoint saved to ../../models/deep_learning\experiment_15.pth
